In [50]:
# Dependencies
from shapely.geometry import shape
import plotly.graph_objs as go
import plotly.express as px
import shapely.geometry
import geopandas as gpd
import pandas as pd
import numpy as np
import requests
import json

### Migration flows by Upper Tier Local Authority

Information about the data can be fount here: https://www.nomisweb.co.uk/sources/census_2021_mig

In [51]:
# Create df for migration into England and Wales
mig_utla_in_df = pd.read_csv("./Resources/MIG004EW_UTLA_IN.csv")
# print(mig_utla_in_df.head())

# Create df for emigration from England and Wales
mig_utla_out_df = pd.read_csv("./Resources/MIG004EW_UTLA_OUT.csv")
# mig_utla_out_df.head()

In [58]:
# Count local authority labels in
utla_authority_in = mig_utla_in_df.groupby(['Upper tier local authorities code', 'Upper tier local authorities label'])['Count'].sum().reset_index()
utla_authority_in = utla_authority_in.sort_values(by='Count', ascending=False)

#Save df to csv file
utla_authority_in.to_csv("./Resources/utla_authority_in.csv", index=False)

# print(utla_authority_in.head)

# Count local authority labels out
utla_authority_out = mig_utla_out_df.groupby(['Migrant UTLA one year ago code', 'Migrant UTLA one year ago label'])['Count'].sum().reset_index()
utla_authority_out = utla_authority_out.sort_values(by='Count', ascending=False)


#Save df to csv file
utla_authority_out.to_csv("./Resources/utla_authority_out.csv", index=False)

# print(utla_authority_out.head)

In [53]:
# Merge DataFrames on the common column
merged_df = pd.merge(utla_authority_in, utla_authority_out, left_on='Upper tier local authorities code', right_on='Migrant UTLA one year ago code', how='outer')

# Fill NaN values with 0
merged_df[['Count_x', 'Count_y']] = merged_df[['Count_x', 'Count_y']].fillna(0)

# Calculate net migration
merged_df['Net Migration'] = merged_df['Count_x'] - merged_df['Count_y']

# Select relevant columns
utla_net_migration = merged_df[['Upper tier local authorities code', 'Upper tier local authorities label', 'Net Migration']]

# Rename columns
utla_net_migration.columns = ['UTLA Code', 'UTLA Label', 'Net Migration']


### Update changes to UTLA's with recent changes to reflect the geoJSON geometry

In [54]:
# Find the rows with the newly combined UTLA's of North and West Hampshire
target_codes = ['E06000062', 'E06000061']
target_rows = utla_net_migration[utla_net_migration['UTLA Code'].isin(target_codes)]

# Create a new row with the combined counts and modified UTLA code and label
combined_count = target_rows['Net Migration'].sum()
new_row = pd.DataFrame({'UTLA Code': ['E10000021'], 'UTLA Label': ['Northamptonshire'], 'Net Migration': [combined_count]})

# Remove the original rows
utla_net_migration = utla_net_migration[~utla_net_migration['UTLA Code'].isin(target_codes)]

# Concatenate the new row with the modified DataFrame
utla_net_migration = pd.concat([utla_net_migration, new_row], ignore_index=True)

# Replace UTLA update Buckinghamshire UTLA code'
utla_net_migration['UTLA Code'] = utla_net_migration['UTLA Code'].replace('E06000060', 'E10000002')

# Save df to csv file
utla_net_migration.to_csv("./Resources/utla_net_migration.csv", index=False)

# Display the first few rows of the net_migration_df
print(utla_net_migration)

     UTLA Code        UTLA Label  Net Migration
0    E10000016              Kent      1416261.0
1    E10000012             Essex      1357508.0
2    E10000014         Hampshire      1259989.0
3    E10000017        Lancashire      1115099.0
4    E10000030            Surrey      1064852.0
..         ...               ...            ...
170  E09000001    City of London         6975.0
171  E06000053   Isles of Scilly         1856.0
172        NaN               NaN    -52532046.0
173        NaN               NaN      -545101.0
174  E10000021  Northamptonshire       702136.0

[175 rows x 3 columns]


### Asign geoJSON data from geoJSON file. geoJSON file download:

https://geoportal.statistics.gov.uk/datasets/ons::counties-and-unitary-authorities-december-2019-boundaries-uk-buc/explore?location=52.040979%2C-3.379571%2C7.36

In [59]:
# Load the CSV data
csv_data = pd.read_csv("./Resources/utla_net_migration.csv")

# Load the GeoDataFrame from the GeoJSON file
geojson_data = gpd.read_file("./Resources/CTYUA_Dec_2019_UGCB_in_the_UK_2022_7196992738345071049.geojson")
geojson_data
# Merge the CSV data with the GeoJSON data based on 'Lower tier local authorities code'
merged_utla = pd.merge(geojson_data, csv_data, left_on='ctyua19cd', right_on='UTLA Code', how='left')

merged_utla = merged_utla.dropna().reset_index()
# print(merged_utla.head())
# merged_utla.dtypes
# na_rows = merged_utla[merged_utla.isna().any(axis=1)]
# na_rows.tail(6)

### Migration Net by County/ ULTA for England and Wales

In [56]:
# Convert the 'geometry' column to GeoJSON format
merged_utla['geometry'] = merged_utla['geometry'].apply(shape)

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(merged_utla, geometry='geometry')

# Create a GeoJSON representation of the GeoDataFrame
geojson_data = gdf.to_crs(epsg=4326).__geo_interface__  # Ensure the GeoJSON is in WGS 84 (EPSG:4326)

# Creating the net flow choropleth map with plotly.express
fig = px.choropleth_mapbox(
    gdf,
    geojson=geojson_data,
    locations=gdf.index,
    color='Net Migration',
    color_continuous_scale="BuPu",
    mapbox_style="carto-positron",
    center={
        "lat": gdf['lat'].mean(),
        "lon": gdf['long'].mean(),
    },
    zoom=5.3,
    opacity=0.7,
    labels={'Net Migration': 'Net Immigration:'},
    custom_data=['UTLA Code', 'UTLA Label', 'Net Migration'],
)

# Update hovertemplate to control the content of the tooltip
fig.update_traces(hovertemplate='<b>%{customdata[0]}</b><br>%{customdata[1]}<br>Net Immigration: %{customdata[2]:,.0f}')

# Customize the layout to show a comma as a thousand separator in color bar tick labels
fig.update_layout(coloraxis_colorbar=dict(
    tickformat=','
))

# Adjust image size 
fig.update_layout(
    width=800,
    height=800
)

# Show or save the figure
fig.show()
# or fig.write_html("your_file.html")